In [1]:
!sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr-por
!python -m pip install pdf2image
!python -m pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (2,098 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 16083

In [2]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import glob
import os
import json
from pdf2image import convert_from_path
import pytesseract # Módulo para a utilização da tecnologia OCR
from pytesseract import Output

In [3]:
from google.colab import drive
drive.mount('/content/drive')

workdir_path = '/content/drive/My Drive/BNDES/Banco de Servicos/'
os.chdir(workdir_path)

Mounted at /content/drive


In [4]:
df1 = pd.read_csv('banco-servicos-banco-de-servicos.csv', sep=';', decimal=',', encoding='UTF-8')

In [5]:
# Download e descompactação dos contratos do Banco de Serviços
lista_projetos = df1['link_projeto'].to_list()
for i in range(len(lista_projetos)):
  r = requests.get(lista_projetos[i])
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall('contratos')

In [6]:
# Selecionar pdfs de contratos
pdf_path = glob.glob('contratos/*')
pdf_contratos = []
for i in range(len(pdf_path)):
  if 'contrato' in (os.path.basename(pdf_path[i])).lower():
    pdf_contratos.append(pdf_path[i])

In [7]:
# Tranformação dos pdfs de contratos em imagems
for i in range(len(pdf_contratos)):
    pages = convert_from_path(pdf_contratos[i])
    for j in range(len(pages)):
      page = pages[j]
      page.save('imagens/' + os.path.basename(pdf_contratos[i]).split('.')[0] + '_{}.jpg'.format(j), 'JPEG')

In [10]:
# Processamento das imagens com o Tesseract
texto = dict()
df2 = pd.DataFrame()
lista_imagens = glob.glob('imagens/*')
for i in range(len(lista_imagens)):
  texto['X'+str(i)] = pytesseract.image_to_data(lista_imagens[i], lang ='por', output_type=Output.DICT)
  d = pd.DataFrame(texto['X'+str(i)])
  df2 = df2.append(d, ignore_index=True)

KeyboardInterrupt: ignored

In [12]:
df2.tail()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
2588646,5,1,19,1,1,3,835,1216,54,8,96,PANTANAL
2588647,2,1,20,0,0,0,196,589,3,685,-1,
2588648,3,1,20,1,0,0,196,589,3,685,-1,
2588649,4,1,20,1,1,0,196,589,3,685,-1,
2588650,5,1,20,1,1,1,196,589,3,685,95,


In [13]:
df2.to_csv('contratos.csv', sep=';', encoding='UTF-8', index=False)